# Installs



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Github/DeepLearning_final

/content/drive/MyDrive/Github/DeepLearning_final


In [13]:
!git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (5/5), 634 bytes | 0 bytes/s, done.
From https://github.com/Jeremylin0904/DeepLearning_final
   12828c7..a37a252  sebas      -> origin/sebas
Updating 12828c7..a37a252
Fast-forward
 train.py | 1 +
 1 file changed, 1 insertion(+)


In [ ]:
!git switch seb

In [3]:
!pip install timm
!pip install av
!pip install joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 45.9 MB/s eta 0:00:00


In [10]:
import re
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from urllib import request
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_video
from torchvision.transforms import v2
import torch
from torch.utils.data import DataLoader, Dataset

import torch.nn.functional as F
from functools import partial
import gc
import av
import joblib

from timm.models.vision_transformer import PatchEmbed

from DataLoading import *
from SiamMae import *
from LabelPropagation import *
from train import *
from utils import *

# Data loading

In [5]:
# Change it in your branch !
root_path = '/content/drive/MyDrive/DL Project/UCF101/UCF-101'

In [8]:
transforms = v2.Compose([
    v2.Resize(size=(224,224), antialias=True),
    v2.Lambda(lambd=lambda x: x/255.0)
])

train_data = UCF101FullVideo(root=root_path, output_format="TCHW",transform=transforms)
train_data.__len__()
train_loader = DataLoader(train_data, 32, shuffle=True, collate_fn=custom_collate, pin_memory=True, num_workers = 1)

# Model training

In [14]:
# Model, optimizer setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sim_mae_vit_small_path16 = sim_mae_vit_small_patch16_dec512d8b
model = sim_mae_vit_small_path16().to(device)
# Change in your branch
folder_logs = '/content/drive/MyDrive/DL Project/Trainings/logs.txt'
folder_model = '/content/drive/MyDrive/DL Project/Trainings'

num_epochs = 100
model = train(model, train_loader, folder_logs, folder_model, num_epochs=num_epochs, lr=1e-4)

/usr/local/lib/python3.10/dist-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


NameError: ignored

# Label propagation

In [ ]:
# Change in your branch
root = '/content/drive/MyDrive/DL Project/DAVIS'

videos_path = os.path.join(root, 'JPEGImages/480p')
labels_path = os.path.join(root, 'Annotations/480p')

## Model loading

In [ ]:
model_path = '/content/drive/MyDrive/DL Project/training/epoch_20.pt'
model = sim_mae_vit_small_patch16_dec512d8b()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.load_state_dict(torch.load(model_path))

## Inference

In [ ]:
name_video = 'bear'
eval_davis(model, name_video, videos_path, labels_path, 20, dino=True)

In [ ]:
video_num = '00001'
show_segm(videos_path, labels_path, name_video, video_num)

## Evaluation

In [ ]:
print(scores_one_vid(videos_path, name_video))